# Quick start
## Import datasets
At first, import our library and datasets from the given path.
Under the given directory, there should be a `data` directory with different datasets. E.g. `/home/AGL/data/cora`.

In [ ]:
import autogl
from autogl.datasets import build_dataset_from_name
cora_dataset = build_dataset_from_name('cora', path = '~/')


## Decide modules
Then, you should decide which models to use.
Here, we use `deepgl` to pre-process graph features, then we use two GNNs to learn the target task, e.g. `GCN` and `GAT`.
We use Simulated annealing algorithm to tune the hyper-parameters of the two GNNs. 
After training, use voting method to ensemble the results of the two GNNs.
Also, you can specify which device to run on.

In [ ]:
import torch
device = torch.device('cuda:5' if torch.cuda.is_available() else 'cpu')
from autogl.solver import AutoNodeClassifier
solver = AutoNodeClassifier(
    feature_module='deepgl',
    graph_models=['gcn', 'gat'],
    hpo_module='anneal',
    ensemble_module='voting',
    device=device
)

## Running
Run the whole process with a certain time limit and show the leaderboard.
You can also get the accuracy by evaluate the predictions.

In [ ]:
solver.fit(cora_dataset, time_limit=3600)
solver.get_leaderboard().show()

from autogl.module.train import Acc
from autogl.solver.utils import get_graph_labels, get_graph_masks

predicted = solver.predict_proba()
label = get_graph_labels(cora_dataset[0])[get_graph_masks(cora_dataset[0], 'test')].cpu().numpy()
print('Test accuracy: ', Acc.evaluate(predicted, label))